<a href="https://colab.research.google.com/github/vishalk31/Tensorflow-Specialization/blob/main/lyrics_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import pandas as pd
#from tensorflow.keras.preprocessing.text import Tokenizer

In [2]:
song_dataframe = pd.read_excel("Song_lyrics.xlsx")

In [3]:
song_dataframe =  song_dataframe[:500]

In [4]:
final_song_Data  = []
for songs in song_dataframe["Lyrics"][:]:
    #print(songs.split("\n"))
    data = " ".join(songs.split("\n"))
    final_song_Data.append(data)
    

In [5]:
final_song_Data =" ".join(final_song_Data)
final_song_Data = final_song_Data.replace(",","")
final_song_Data = final_song_Data.replace("!","")
final_song_Data = final_song_Data.replace(".","")
final_song_Data = final_song_Data.split(" ")

In [6]:
final_song_Data = [values for values in final_song_Data if len(values)>1 ]

In [7]:
counter = 0
train_set = []
train_label = []
train_data_avail = ""
for index,lyrics in enumerate(final_song_Data[:len(final_song_Data)-1]):
    #print(index,lyrics)
    if counter != 100 :
        train_data_avail = train_data_avail+" "+lyrics
        train_set.append(train_data_avail)
        train_label.append(final_song_Data[index+1])
        
        
    elif counter == 100:
        train_data_avail = train_data_avail+" "+lyrics
        train_set.append(train_data_avail)
        train_label.append(final_song_Data[index+1])
        train_data_avail = ""
        counter = 0
    
    counter = counter+1
        
    

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
#corpus = ['Text processing is necessary.', 'Text processing is necessary and important.', 'Text processing is easy.']
vectorizer = CountVectorizer(ngram_range=(2, 2), max_features=100)
X = vectorizer.fit_transform([" ".join(final_song_Data)])
#print(vectorizer.get_feature_names())

In [9]:
train_input = vectorizer.transform(train_set).toarray()

In [10]:
len(train_input[0])

100

In [11]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
train_output = le.fit_transform(train_label)

In [12]:
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np 

In [13]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(train_input), 16, input_length=len(train_input[0])),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 16)           3414528   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               12544     
 l)                                                              
                                                                 
 dense (Dense)               (None, 24)                1560      
                                                                 
 dense_1 (Dense)             (None, 1)                 25        
                                                                 
Total params: 3,428,657
Trainable params: 3,428,657
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(train_input, train_output,epochs=10)

Epoch 1/10
6669/6669 [==============================] - 542s 81ms/step - loss: -64613308.0000 - accuracy: 4.6859e-05
Epoch 2/10
6669/6669 [==============================] - 548s 82ms/step - loss: -366931136.0000 - accuracy: 4.6859e-05
Epoch 3/10
6669/6669 [==============================] - 559s 84ms/step - loss: -891699648.0000 - accuracy: 4.6859e-05
Epoch 4/10
6669/6669 [==============================] - 557s 84ms/step - loss: -1630591872.0000 - accuracy: 4.6859e-05
Epoch 5/10
6669/6669 [==============================] - 556s 83ms/step - loss: -2581442816.0000 - accuracy: 4.6859e-05
Epoch 6/10
6669/6669 [==============================] - 559s 84ms/step - loss: -3742130688.0000 - accuracy: 4.6859e-05
Epoch 7/10
6669/6669 [==============================] - 562s 84ms/step - loss: -5111630336.0000 - accuracy: 4.6859e-05
Epoch 8/10
6669/6669 [==============================] - 570s 85ms/step - loss: -6689297920.0000 - accuracy: 4.6859e-05
Epoch 9/10
6669/6669 [==============================

In [16]:
model.save("lyrics.h5")      

In [17]:
import pickle
with open('vectorizer.pk', 'wb') as fin:
  pickle.dump(vectorizer, fin)